In [8]:
%cd /Users/typhaine/Documents/Doc Gorilla/WhereIsMyMNIST/notebooks

/Users/typhaine/Documents/Doc Gorilla/WhereIsMyMNIST/notebooks


In [9]:
import os
import imageio
import random
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import IterableDataset, ChainDataset, DataLoader


import pygame
from psychopy import visual, core, event, gui, data

from what import WhatShift, WhatBackground, WhatNet, WhatTrainer, What, train, test, MNIST, MotionCloudNoise
import MotionClouds as mc

from main import init
args = init(filename='../data/2019-06-12') # pas de drop out!
args


{'w': 28,
 'minibatch_size': 100,
 'train_batch_size': 50000,
 'test_batch_size': 10000,
 'noise_batch_size': 1000,
 'mean': 0.1307,
 'std': 0.3081,
 'what_offset_std': 15,
 'what_offset_max': 25,
 'N_pic': 128,
 'offset_std': 30,
 'offset_max': 34,
 'noise': 0.75,
 'contrast': 0.7,
 'sf_0': 0.1,
 'B_sf': 0.1,
 'do_mask': True,
 'N_theta': 6,
 'N_azimuth': 24,
 'N_eccentricity': 10,
 'N_phase': 2,
 'rho': 1.41,
 'bias_deconv': True,
 'p_dropout': 0.0,
 'dim1': 1000,
 'dim2': 1000,
 'lr': 0.005,
 'do_adam': True,
 'bn1_bn_momentum': 0.5,
 'bn2_bn_momentum': 0.5,
 'momentum': 0.3,
 'epochs': 60,
 'num_processes': 1,
 'no_cuda': False,
 'log_interval': 100,
 'verbose': 1,
 'filename': '../data/2019-06-12',
 'seed': 2019,
 'N_cv': 10,
 'do_compute': True,
 'save_model': True}

In [11]:
class ChainDataset(IterableDataset):
    def __init__(self, datasets: Iterable[Dataset]):
        self.datasets = datasets

    def __iter__(self):
        for d in self.datasets:
            assert isinstance(d, IterableDataset), "ChainDataset only supports IterableDataset"
            for x in d:
                yield x

    def __len__(self):
        total = 0
        for d in self.datasets:
            assert isinstance(d, IterableDataset), "ChainDataset only supports IterableDataset"
            # Cannot verify that all self.datasets are Sized
            total += len(d)  # type: ignore
        return total

NameError: name 'Iterable' is not defined

In [ ]:
transform_1=transforms.Compose([
                               WhatShift(args,
                                         i_offset=random.randint(-50,50), 
                                         j_offset=random.randint(-50,50)),
                               WhatBackground(contrast = args.contrast,
                                              noise=0.5, 
                                              sf_0=args.sf_0, 
                                              B_sf=args.B_sf),
                               transforms.ToTensor(),
                               transforms.Normalize((args.mean,), (args.std,))
                           ])

dataset_train_1 = MNIST('../data',
                        train=True,
                        download=True,
                        transform=transform_1,
                        )
transform_2=transforms.Compose([
                               WhatShift(args,
                                         i_offset=random.randint(-50,0), 
                                         j_offset=random.randint(-50,0)),
                               WhatBackground(contrast = args.contrast,
                                              noise=0.5, 
                                              sf_0=args.sf_0, 
                                              B_sf=args.B_sf),
                               transforms.ToTensor(),
                               transforms.Normalize((args.mean,), (args.std,))
                           ])

dataset_train_2 = MNIST('../data',
                        train=True,
                        download=True,
                        transform=transform_2,
                        )

transform_3 =transforms.Compose([
                               WhatShift(args,
                                         i_offset=random.randint(-50,0), 
                                         j_offset=random.randint(0,50)),
                               WhatBackground(contrast = args.contrast,
                                              noise=0.5, 
                                              sf_0=args.sf_0, 
                                              B_sf=args.B_sf),
                               transforms.ToTensor(),
                               transforms.Normalize((args.mean,), (args.std,))
                           ])

dataset_train_3 = MNIST('../data',
                        train=True,
                        download=True,
                        transform=transform_3,
                        )

transform_4 =transforms.Compose([
                               WhatShift(args,
                                         i_offset=random.randint(0,50), 
                                         j_offset=random.randint(-50,0)),
                               WhatBackground(contrast = args.contrast,
                                              noise=0.5, 
                                              sf_0=args.sf_0, 
                                              B_sf=args.B_sf),
                               transforms.ToTensor(),
                               transforms.Normalize((args.mean,), (args.std,))
                           ])

dataset_train_4 = MNIST('../data',
                        train=True,
                        download=True,
                        transform=transform_4,
                        )
    
train_loader = DataLoader(ChainDataset([dataset_train_1,dataset_train_2]),
                                         batch_size=2,
                                         shuffle=False)

data,label = next(iter(train_loader))

In [7]:
print(data.shape)
print(label[0].shape)
print(label)
print(data)

AttributeError: module 'psychopy.data' has no attribute 'shape'

In [119]:
#save = torch.maximum(data1[0],(data2[0],data3[0]))
temp = torch.maximum(data)

#temp = np.repeat(temp,60).reshape(128,128,60)
print(temp.shape)
#temp = np.tile((temp),60).reshape(128,128,60)
plt.imshow(temp[1,:,:])

TypeError: maximum() missing 1 required positional arguments: "other"